# Credit EDA Case Study

### The porblem statement of this case study can be found at:

https://learn.upgrad.com/v/course/258/session/36098/segment/192309

## Table of Contents:

  1. [Import Libraries and setup environment](#Import)
  2. [Read csv data file and create Dataframe](#ReadData)
  3. [Check dataframe size and basic data quality](#DataChecks)
  4. [Missing data analysis and deletion of high % missing data columns](#MissingData)
  5. [Data Imputation](#DataImputation)
   - [DAYS_LAST_PHONE_CHANGE](#DataImputation_1)
   - [CNT_FAM_MEMBERS](#DataImputation_2)
   - [NAME_TYPE_SUITE](#DataImputation_3)
   - [AMT_GOODS_PRICE](#DataImputation_4)
   - [AMT_REQ_CREDIT_BUREAU_* columns](#DataImputation_5)
   - [client's social surroundings with observable columns](#DataImputation_6)
  6. [Inspect Datatypes of Variables](#InspectDatatype)
   - [Object dtype](#Object)
   - [int64 dtype](#Int)
      * [Data fix for DAYS_BIRTH, DAYS_EMPLOYED,  DAYS_ID_PUBLISH](#Fix_1)
   - [float64 dtype](#Float)
      * [Dtype change](#Fix_2)
  7. [Outliers in the numerical data](#Outliers)
   - [AMT_INCOME_TOTAL](#AMT_INCOME_TOTAL)
   - [AMT_ANNUITY](#AMT_ANNUITY)
   - [AMT_GOODS_PRICE](#AMT_GOODS_PRICE)
   - [CNT_FAM_MEMBERS](#CNT_FAM_MEMBERS)
  8. [Binnning the continous variables](#Binning)
   - [DAYS_BIRTH](#DAYS_BIRTH)
   - [DAYS_EMPLOYED](#DAYS_EMPLOYED)
  9. [Export data to csv for futher analysis](#Export)






In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

<a id='Import'></a>
## Import all the required libraries

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import plotly.plotly as py
from plotly import tools
import plotly.graph_objs as go
sns.set(style="darkgrid")
%matplotlib inline
#importing plotly and cufflinks in offline mode
import cufflinks as cf
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [0]:
# Create a class color for setting print formatting
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'


In [0]:
# Function to display plotly offline plots in Juputer notebook cells
def enable_plotly_in_cell():
  import IPython
  from plotly.offline import init_notebook_mode
  display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
  init_notebook_mode(connected=False)

In [0]:
#Setup the display options so that all the columns are displayed on the screen

pd.options.display.max_columns = 150

<a id='ReadData'></a>
## Read csv data file and create Dataframe


In [0]:
#Create a dataframe df with all the application_data for further processing

#df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/data/application_data.csv")

In [0]:
df = pd.read_csv("application_data.csv")

In [0]:
#pp.ProfileReport(df)

<a id='DataChecks'></a>
## Check dataframe size and basic data quality

In [0]:
# Display shape of the dataframe


print(color.BOLD+color.BLUE+'Shape of the dataframe df : {}'.format(df.shape) +color.END)

In [0]:
# Display first 5 rows of the dataframe df to have a quick view at the data

df.head()

In [0]:
df.describe()

In [0]:
#df[df['TARGET'] ==1]

<a id='MissingData'></a>
## Missing data analysis and deletion of high % missing data columns

In [0]:
#Before moving ahead with our EDA, we need to check dataframe for the missing data.
#In the case where the column has more than 50% of the data missing,
#it would be adivisable to remove that column as the analysis of columns with more than 50% data would not give us correct results

#We will display total missing records for each column and find the % of the missing values which can help use for our next step.

Total_missing = df.isnull().sum().sort_values(ascending = False)
Total_missing_Perc = (100*df.isnull().sum()/df.shape[0]).sort_values(ascending = False)
df_missing_values = pd.concat([Total_missing,Total_missing_Perc], axis=1, keys=['Total_missing_values', 'Percent_missing_values'])
df_missing_values.head(25)

In [0]:
# let us check how many columns have more than 50% data missing



print(color.BOLD+ color.BLUE+'Total no of columns with missing values more than 50% : {}'.format(df_missing_values[df_missing_values['Percent_missing_values'] >= 50].shape[0])+ color.END)

In [0]:
#Create a dataframe with all columns with data misisng < 50% for our  further analsysis

df_cleaned = df.loc[:,(100*df.isnull().sum()/df.shape[0]).sort_values(ascending = False) < 50]

In [0]:
df_cleaned.head()

In [0]:
#Check no of missing values for  each record
cnt = (df_cleaned.isnull().sum(axis= 1).sort_values(ascending = False)).values
collections.Counter(cnt)

print(color.BOLD+ color.BLUE+'Total no of records with missing values for  more than 10 columns : {}'.format(df_cleaned[df_cleaned.isnull().sum(axis= 1) > 10].shape[0])+ color.END)


<a id ='DataImputation'></a>
## Data imputation

Data imputation is used for replacing missing values with appropriate values which can aid in further data analysis.



In [0]:
#Check missing values
#we need to impute values for columns with low missing %
# we will consider 15% and lower for the data imputation activity
df_missing_values[(df_missing_values['Percent_missing_values'] < 15) 
                  & (df_missing_values['Percent_missing_values'] > 0)]
                                                                                                                                                                                    

<a id = 'DataImputation_1' ></a>
### Data imputation for DAYS_LAST_PHONE_CHANGE

In [0]:
# we will plot histogram to check value frequency
# as well as Box plot to check the central tendency for DAYS_LAST_PHONE_CHANGE data
enable_plotly_in_cell()

fig = tools.make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                          shared_yaxes=False, vertical_spacing=0.001)


trace0 = go.Histogram(
    x=df_cleaned['DAYS_LAST_PHONE_CHANGE'],
    nbinsx = 20,
    name = "Count"
)
trace1 = go.Box(
    y=df_cleaned['DAYS_LAST_PHONE_CHANGE'],
    boxpoints = 'outliers',
    name = "Days"
)
data = [trace0, trace1]
layout = go.Layout(
    title = "Plots for DAYS_LAST_PHONE_CHANGE"
)
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig['layout']['xaxis1'].update(title='Days')
fig['layout']['yaxis1'].update(title='No of records')
fig['layout']['yaxis2'].update(title='Days')
fig['layout'].update(height=600, width=800, title='Plots for DAYS_LAST_PHONE_CHANGE')
cf.iplot(fig)

# Let's check how many values for are null for column 'DAYS_LAST_PHONE_CHANGE'

clmn = 'DAYS_LAST_PHONE_CHANGE'

Total_null_values = df_cleaned['DAYS_LAST_PHONE_CHANGE'].isnull().sum()

print(color.BOLD + color.BLUE + 'Total no of null values for column {0} : {1}'.format(clmn,Total_null_values ) + color.END)

#check outliers for the DAYS_LAST_PHONE_CHANGE

Total_no_of_outliers = df[df_cleaned['DAYS_LAST_PHONE_CHANGE'] < -3500].shape[0]

print(color.BOLD + color.BLUE + 'Total no of outliers for column {0} : {1}'.format(clmn,Total_no_of_outliers ) + color.END)

In [0]:
# As DAYS_LAST_PHONE_CHANGE is a discrete variable, we will replace all NaN values with 'median' value as that value is most common value


df_cleaned['DAYS_LAST_PHONE_CHANGE'].fillna(df_cleaned['DAYS_LAST_PHONE_CHANGE'].median(), inplace = True)
df_cleaned['DAYS_LAST_PHONE_CHANGE'].isnull().sum()

print(color.BOLD + color.BLUE + 'Total no of missing values replaced by median value for column {0} : {1}'.format(clmn,Total_null_values ) + color.END)


<a id = 'DataImputation_2' ></a>
### Data imputation for CNT_FAM_MEMBERS

In [0]:
# we will plot histogram to check value frequency
# as well as Box plot to check the central tendency for CNT_FAM_MEMBERS data
enable_plotly_in_cell()

fig = tools.make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                          shared_yaxes=False, vertical_spacing=0.001)


trace0 = go.Histogram(
    x=df_cleaned['CNT_FAM_MEMBERS'],
    nbinsx = 20,
    name = "Count"
)
trace1 = go.Box(
    y=df_cleaned['CNT_FAM_MEMBERS'],
    boxpoints = 'outliers',
    name = "No of Family Members"
)
data = [trace0, trace1]
layout = go.Layout(
    title = "Plots for CNT_FAM_MEMBERS"
)
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig['layout']['xaxis1'].update(title='No of Family Members')
fig['layout']['yaxis1'].update(title='No of records')
fig['layout']['yaxis2'].update(title='No of Family Members')
fig['layout'].update(height=600, width=800, title='Plots for CNT_FAM_MEMBERS')
cf.iplot(fig)

# Let's check how many values for are null for column 'CNT_FAM_MEMBERS'

clmn = 'CNT_FAM_MEMBERS'

Total_null_values = df_cleaned['CNT_FAM_MEMBERS'].isnull().sum()

print(color.BOLD + color.BLUE + 'Total no of null values for column {0} : {1}'.format(clmn,Total_null_values ) + color.END)

#check outliers for the CNT_FAM_MEMBERS

Total_no_of_outliers = df[df_cleaned['CNT_FAM_MEMBERS'] > 4].shape[0]

print(color.BOLD + color.BLUE + 'Total no of outliers for column {0} : {1}'.format(clmn,Total_no_of_outliers ) + color.END)

In [0]:
# As CNT_FAM_MEMBERS is a discrete variable, we will replace all NaN values with 'median' value as that value is most common value

df_cleaned['CNT_FAM_MEMBERS'].fillna(df_cleaned['CNT_FAM_MEMBERS'].median(), inplace = True)
df_cleaned['CNT_FAM_MEMBERS'].isnull().sum()

print(color.BOLD + color.BLUE + 'Total no of missing values replaced by median value for column {0} : {1}'.format(clmn,Total_null_values ) + color.END)



<a id = 'DataImputation_3' ></a>
### Data imputation for NAME_TYPE_SUITE

In [0]:
#We will find value counts for NAME_TYPE_SUITE categories. 
#As this is categorical variable will impute the missing data with high frequency category 
df_cleaned['NAME_TYPE_SUITE'].value_counts()

In [0]:
# Let's check how many values for are null for column 'NAME_TYPE_SUITE'

clmn = 'NAME_TYPE_SUITE'

Total_null_values = df_cleaned['NAME_TYPE_SUITE'].isnull().sum()

print(color.BOLD + color.BLUE + 'Total no of null values for column {0} : {1}'.format(clmn,Total_null_values ) + color.END)


In [0]:
# As 'Unaccompanied' is the major value for NAME_TYPE_SUITE, we will replace all NaN values with 'Unaccompanied'

df_cleaned['NAME_TYPE_SUITE'].fillna('Unaccompanied', inplace = True)
df_cleaned['NAME_TYPE_SUITE'].isnull().sum()

print(color.BOLD + color.BLUE + 'Total no of missing values replaced by "Unaccompanied" value for column {0} : {1}'.format(clmn,Total_null_values ) + color.END)



<a id = 'DataImputation_4' ></a>
### Data imputation for AMT_GOODS_PRICE

In [0]:
# we will plot histogram to check value frequency
# as well as Box plot to check the central tendency for DAYS_LAST_PHONE_CHANGE data
enable_plotly_in_cell()

fig = tools.make_subplots(rows=1, cols=2, specs=[[{}, {}]], shared_xaxes=True,
                          shared_yaxes=False, vertical_spacing=0.001)


trace0 = go.Histogram(
    x=df_cleaned['AMT_GOODS_PRICE'],
    nbinsx = 20,
    name = "Count"
)
trace1 = go.Box(
    y=df_cleaned['AMT_GOODS_PRICE'],
    boxpoints = 'outliers',
    name = "No of Family Members"
)
data = [trace0, trace1]
layout = go.Layout(
    title = "Plots for AMT_GOODS_PRICE"
)
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 2)
fig['layout']['xaxis1'].update(title='AMT_GOODS_PRICE')
fig['layout']['yaxis1'].update(title='No of records')
fig['layout']['yaxis2'].update(title='AMT_GOODS_PRICE')
fig['layout'].update(height=600, width=800, title='Plots for AMT_GOODS_PRICE')
cf.iplot(fig)

# Let's check how many values for are null for column 'AMT_GOODS_PRICE'

clmn = 'AMT_GOODS_PRICE'

Total_null_values = df_cleaned['AMT_GOODS_PRICE'].isnull().sum()

print(color.BOLD + color.BLUE + 'Total no of null values for column {0} : {1}'.format(clmn,Total_null_values ) + color.END)

In [0]:
Total_no_of_outliers = df[df_cleaned['AMT_GOODS_PRICE'] > 1341000].shape[0]

print(color.BOLD + color.BLUE + 'Total no of outliers for column {0} : {1}'.format(clmn,Total_no_of_outliers ) + color.END)

In [0]:
# As AMT_GOODS_PRICE is a discrete variable, we will replace all NaN values with 'mean' value 

df_cleaned['AMT_GOODS_PRICE'].fillna(df_cleaned['AMT_GOODS_PRICE'].mean(), inplace = True)
df_cleaned['AMT_GOODS_PRICE'].isnull().sum()

print(color.BOLD + color.BLUE + 'Total no of missing values replaced by average value for column {0} : {1}'.format(clmn,Total_null_values ) + color.END)


<a id = 'DataImputation_5' ></a>
### Data imputation for AMT_REQ_CREDIT_BUREAU_* columns

In [0]:
# We need to check if all AMT_REQ_CREDIT_BUREAU_* columns have null values for same records
missing_val = df_cleaned[(df_cleaned['AMT_REQ_CREDIT_BUREAU_QRT'].isnull() ) &
          (df_cleaned['AMT_REQ_CREDIT_BUREAU_YEAR'].isnull() ) &
           (df_cleaned['AMT_REQ_CREDIT_BUREAU_WEEK'].isnull() ) &
           (df_cleaned['AMT_REQ_CREDIT_BUREAU_MON'].isnull() ) &
           (df_cleaned['AMT_REQ_CREDIT_BUREAU_DAY'].isnull() ) &
          (df_cleaned['AMT_REQ_CREDIT_BUREAU_HOUR'].isnull() )].shape[0]

print(color.BOLD + color.BLUE + 'Total no of missing values : {0} '.format(missing_val ) + color.END)


In [0]:
#we need to find value which has highest count for AMT_REQ_CREDIT_BUREAU_QRT
#this value can be used for further imputing
Qrt_cnt = df_cleaned['AMT_REQ_CREDIT_BUREAU_QRT'].value_counts().nlargest(1)

print(color.BOLD + color.BLUE + 'The highest no of values for AMT_REQ_CREDIT_BUREAU_QRT is {0} with count: {1}'.format(Qrt_cnt.index[0],Qrt_cnt[0] ) + color.END)


#we need to find value which has highest count for AMT_REQ_CREDIT_BUREAU_YEAR
Year_cnt = df_cleaned['AMT_REQ_CREDIT_BUREAU_YEAR'].value_counts().nlargest(1)

print(color.BOLD + color.BLUE + 'The highest no of values for AMT_REQ_CREDIT_BUREAU_YEAR is {0} with count: {1}'.format(Year_cnt.index[0],Year_cnt[0] ) + color.END)


#we need to find value which has highest count for AMT_REQ_CREDIT_BUREAU_WEEK
WEEK_cnt = df_cleaned['AMT_REQ_CREDIT_BUREAU_WEEK'].value_counts().nlargest(1)

print(color.BOLD + color.BLUE + 'The highest no of values for AMT_REQ_CREDIT_BUREAU_WEEK is {0} with count: {1}'.format(WEEK_cnt.index[0],WEEK_cnt[0] ) + color.END)


#we need to find value which has highest count for AMT_REQ_CREDIT_BUREAU_MON
MON_cnt = df_cleaned['AMT_REQ_CREDIT_BUREAU_MON'].value_counts().nlargest(1)

print(color.BOLD + color.BLUE + 'The highest no of values for AMT_REQ_CREDIT_BUREAU_MON is {0} with count: {1}'.format(MON_cnt.index[0],MON_cnt[0] ) + color.END)


#we need to find value which has highest count for AMT_REQ_CREDIT_BUREAU_DAY
DAY_cnt = df_cleaned['AMT_REQ_CREDIT_BUREAU_DAY'].value_counts().nlargest(1)

print(color.BOLD + color.BLUE + 'The highest no of values for AMT_REQ_CREDIT_BUREAU_DAY is {0} with count: {1}'.format(DAY_cnt.index[0],DAY_cnt[0] ) + color.END)


#we need to find value which has highest count for AMT_REQ_CREDIT_BUREAU_HOUR
HOUR_cnt = df_cleaned['AMT_REQ_CREDIT_BUREAU_HOUR'].value_counts().nlargest(1)

print(color.BOLD + color.BLUE + 'The highest no of values for AMT_REQ_CREDIT_BUREAU_HOUR is {0} with count: {1}'.format(HOUR_cnt.index[0],HOUR_cnt[0] ) + color.END)





**It seems that default value, in absense of any valid value is 0.0.
so we will impute these columns with 0.0**

In [0]:
AMT_REQ_CREDIT_BUREAU_columns = ['AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR','AMT_REQ_CREDIT_BUREAU_WEEK',
                                'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_HOUR']


for col in AMT_REQ_CREDIT_BUREAU_columns:
    df_cleaned[col].fillna(0.0, inplace = True)


print(color.BOLD + color.BLUE + 'Total no of missing values replaced by 0.0 : {0}'.format(missing_val ) + color.END)

    
        

<a id = 'DataImputation_6' ></a>
### Data imputation for 
  - **OBS_30_CNT_SOCIAL_CIRCLE**
  - **DEF_30_CNT_SOCIAL_CIRCLE**
  - **OBS_60_CNT_SOCIAL_CIRCLE**
  - **DEF_60_CNT_SOCIAL_CIRCLE**
  
**As these are discete variables but the dtype is float,so we need to change the dtype to int.
for this purpose we need to impute the missing values**

In [0]:
# We need to check if all CNT_SOCIAL_CIRCLE columns have null values for same records
missing_val = df_cleaned[(df_cleaned['OBS_30_CNT_SOCIAL_CIRCLE'].isnull() ) &
          (df_cleaned['DEF_30_CNT_SOCIAL_CIRCLE'].isnull() ) &
           (df_cleaned['DEF_60_CNT_SOCIAL_CIRCLE'].isnull() ) &
           (df_cleaned['OBS_60_CNT_SOCIAL_CIRCLE'].isnull() ) ].shape[0]

print(color.BOLD + color.BLUE + 'Total no of missing values : {0} '.format(missing_val ) + color.END)

In [0]:
#we need to find value which has highest count for OBS_30_CNT_SOCIAL_CIRCLE
#this value can be used for further imputing
OBS_30 = df_cleaned['OBS_30_CNT_SOCIAL_CIRCLE'].value_counts().nlargest(1)

print(color.BOLD + color.BLUE + 'The highest no of values for OBS_30_CNT_SOCIAL_CIRCLE is {0} with count: {1}'.format(OBS_30.index[0],OBS_30[0] ) + color.END)


#we need to find value which has highest count for DEF_30_CNT_SOCIAL_CIRCLE
DEF_30 = df_cleaned['DEF_30_CNT_SOCIAL_CIRCLE'].value_counts().nlargest(1)

print(color.BOLD + color.BLUE + 'The highest no of values for DEF_30_CNT_SOCIAL_CIRCLE is {0} with count: {1}'.format(DEF_30.index[0],DEF_30[0] ) + color.END)


#we need to find value which has highest count for OBS_60_CNT_SOCIAL_CIRCLE
OBS_60 = df_cleaned['OBS_60_CNT_SOCIAL_CIRCLE'].value_counts().nlargest(1)

print(color.BOLD + color.BLUE + 'The highest no of values for OBS_60_CNT_SOCIAL_CIRCLE is {0} with count: {1}'.format(OBS_60.index[0],OBS_60[0] ) + color.END)


#we need to find value which has highest count for DEF_60_CNT_SOCIAL_CIRCLE
DEF_60 = df_cleaned['DEF_60_CNT_SOCIAL_CIRCLE'].value_counts().nlargest(1)

print(color.BOLD + color.BLUE + 'The highest no of values for DEF_60_CNT_SOCIAL_CIRCLE is {0} with count: {1}'.format(DEF_60.index[0],DEF_60[0] ) + color.END)


**It seems that default value, in absense of any valid value is 0.0.
so we will impute these columns with 0.0**

In [0]:
CNT_SOCIAL_CIRCLE_columns = ['OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE',
                                 'OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE']


for col in CNT_SOCIAL_CIRCLE_columns:
    df_cleaned[col].fillna(0.0, inplace = True)


print(color.BOLD + color.BLUE + 'Total no of missing values replaced by 0.0 : {0}'.format(missing_val ) + color.END)


<a id = 'InspectDatatype' ></a>


### Inspect Datatypes of Variables

In [0]:
#check all dtype counts
df_cleaned.dtypes.value_counts()

<a id = 'Object' ></a>
### Inspect columns with object dtype

In [0]:
#print columns for object dtype
df_cleaned.select_dtypes('object').columns

In [0]:
#print the dtype = object records so that we can have quick look at data to check if any data issues
df_cleaned[df_cleaned.select_dtypes('object').columns].head()

**<span style="color:blue">
    As dtype = object columns are categorical variables, they do not need any dtype change
    </span>**

<a id = 'Int' ></a>
### Inspect columns with int64 dtype

In [0]:
#print columns for int64 dtype
df_cleaned.select_dtypes('int64').columns

In [0]:
#print the dtype = int64 records so that we can have quick look at data to check if any data issues
df_cleaned[df_cleaned.select_dtypes('int64').columns].head()

<a id = 'Fix_1'></a>
**<span style="color:blue">
    As DAYS_BIRTH,DAYS_EMPLOYED, DAYS_ID_PUBLISH are descrete values. 
    we can change them to +values as -ve values do not make any sense and can affect future analysis
    </span>**

In [0]:
#As DAYS_BIRTH,DAYS_EMPLOYED, DAYS_ID_PUBLISH are descrete values, 
#we can change them to +values as -ve values do not make any sense

no_of_rec = df_cleaned[df_cleaned['DAYS_BIRTH'] >= 0].shape[0]
print(color.BOLD + color.BLUE + 'Total no of records with + value for DAYS_BIRTH  : {0}'.format(no_of_rec ) + color.END)
df_cleaned['DAYS_BIRTH']  = np.abs(df_cleaned['DAYS_BIRTH'])



no_of_rec = df_cleaned[df_cleaned['DAYS_EMPLOYED'] >= 0].shape[0]
print(color.BOLD + color.BLUE + 'Total no of records with + value for DAYS_EMPLOYED  : {0}'.format(no_of_rec ) + color.END)
df_cleaned['DAYS_EMPLOYED']  = np.abs(df_cleaned['DAYS_EMPLOYED'])



no_of_rec = df_cleaned[df_cleaned['DAYS_ID_PUBLISH'] >= 0].shape[0]
print(color.BOLD + color.BLUE + 'Total no of records with + value for DAYS_ID_PUBLISH  : {0}'.format(no_of_rec ) + color.END)
df_cleaned['DAYS_ID_PUBLISH']  = np.abs(df_cleaned['DAYS_ID_PUBLISH'])


print(color.BOLD + color.BLUE + 'Changed columns DAYS_BIRTH ,DAYS_EMPLOYED,DAYS_ID_PUBLISH values from -ve to absolute' + color.END)


In [0]:
# for NAME_INCOME_TYPE = Pensioner, DAYS_EMPLOYED should be 365243 which is a default value.
# This isa data quality issue
#df_cleaned[df_cleaned['DAYS_EMPLOYED'] >= 0]
df_cleaned[df_cleaned['NAME_INCOME_TYPE'] == 'Pensioner']['DAYS_EMPLOYED'].value_counts()

<a id = 'Float' ></a>
### Inspect columns with float64 dtype

In [0]:
#print columns for float64 dtype
df_cleaned.select_dtypes('float64').columns

In [0]:
#print the dtype = float64 records so that we can have quick look at data to check if any data issues
df_cleaned[df_cleaned.select_dtypes('float64').columns].head()

<a id = 'Fix_2'></a>
### Dtype change for 
  - **DAYS_REGISTRATION**
  - **CNT_FAM_MEMBERS**
  - **OBS_30_CNT_SOCIAL_CIRCLE**
  - **DEF_30_CNT_SOCIAL_CIRCLE**
  - **OBS_60_CNT_SOCIAL_CIRCLE**
  - **DEF_60_CNT_SOCIAL_CIRCLE**
  - **DAYS_LAST_PHONE_CHANGE**

In [0]:
#DAYS_REGISTRATION is a descrete variable so it can be changed from float to int
#also the value should be changed to absolute

#Step 1: change dtype

col_list = ['DAYS_REGISTRATION','CNT_FAM_MEMBERS','DAYS_LAST_PHONE_CHANGE',
            'OBS_30_CNT_SOCIAL_CIRCLE','DEF_30_CNT_SOCIAL_CIRCLE','OBS_60_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE' ]
for col in col_list:
    df_cleaned[col] = df_cleaned[col].astype('int')
    
print(color.BOLD + color.BLUE + 'Changed {0} columns dataype to int'.format(len(col_list)) + color.END)

#Step 2: change values from negative to absolute
no_of_rec = df_cleaned[df_cleaned['DAYS_REGISTRATION'] >= 0].shape[0]
print(color.BOLD + color.BLUE + 'Total no of records with + value for DAYS_REGISTRATION  : {0}'.format(no_of_rec ) + color.END)
df_cleaned['DAYS_REGISTRATION']  = np.abs(df_cleaned['DAYS_REGISTRATION'])
df_cleaned['DAYS_LAST_PHONE_CHANGE']  = np.abs(df_cleaned['DAYS_LAST_PHONE_CHANGE'])


print(color.BOLD + color.BLUE + 'Changed columns DAYS_REGISTRATION & DAYS_LAST_PHONE_CHANGE dataype to int as well as values from -ve to absolute' + color.END)


<a id = 'Outliers'></a>

## Outliers in the Numerical Data

**We will check numerical variables for the outliers, outliers can affect, the sample statistics and might affect future analysis.**

**We will consider below variables for the outlier analysis**

  - AMT_INCOME_TOTAL
  - AMT_ANNUITY
  - AMT_GOODS_PRICE
  - CNT_FAM_MEMBERS
  
  

In [0]:
df_cleaned[['AMT_INCOME_TOTAL', 'AMT_ANNUITY', 'AMT_GOODS_PRICE','CNT_FAM_MEMBERS']].describe()

**We have selected above 4 columns for outlier analysis by looking at Minimum/maxiumum values as well as the mean and standard deviation values**


**as the above columns have larger standard deviation we would like to reduce the standard deviation to reduce bias in future analsysis and inferences**

<a id = 'AMT_INCOME_TOTAL'></a>

### Outlier analysis for AMT_INCOME_TOTAL

In [0]:
enable_plotly_in_cell()
clmn= 'AMT_INCOME_TOTAL'
df_cleaned['AMT_INCOME_TOTAL'].iplot(kind="box", boxpoints = 'outliers', theme="white", title="AMT_INCOME_TOTAL", yTitle='Price in $')


In [0]:
#Lets calculate the upper bound and lower bounds for outliers
q1, q3= np.percentile(df_cleaned['AMT_INCOME_TOTAL'].dropna(),[25,75])
iqr = q3 - q1
lower_bound = q1 -(1.5 * iqr) 
upper_bound = q3 +(1.5 * iqr) 

In [0]:
Total_no_of_outliers = df_cleaned[(df_cleaned['AMT_INCOME_TOTAL'] > upper_bound) | (df_cleaned['AMT_INCOME_TOTAL'] < lower_bound) ].shape[0]

print(color.BOLD + color.BLUE + 'Lower bound for outliers : {0} & Upper bound : {1}'.format(lower_bound,upper_bound ) + color.END)

print(color.BOLD + color.BLUE + 'Total no of outliers for column {0} : {1}'.format(clmn,Total_no_of_outliers ) + color.END)

In [0]:
#calculate % of outliers in the data
100*Total_no_of_outliers/df_cleaned.shape[0]

In [0]:
#Delete the AMT_INCOME_TOTAL outliers from the dataset
df_cleaned = df_cleaned[(df_cleaned['AMT_INCOME_TOTAL'] <= upper_bound) & (df_cleaned['AMT_INCOME_TOTAL'] >= lower_bound )]

<a id = 'AMT_ANNUITY'></a>

### Outlier analysis for AMT_ANNUITY

In [0]:
enable_plotly_in_cell()
clmn = 'AMT_ANNUITY'
df_cleaned['AMT_ANNUITY'].iplot(kind="box", boxpoints = 'outliers', theme="white", title="AMT_ANNUITY",
                                 yTitle='Price in $')




In [0]:
#Lets calculate the upper bound and lower bounds for outliers
q1, q3= np.percentile(df_cleaned['AMT_ANNUITY'].dropna(),[25,75])
iqr = q3 - q1
lower_bound = q1 -(1.5 * iqr) 
upper_bound = q3 +(1.5 * iqr) 

In [0]:

Total_no_of_outliers = df_cleaned[(df_cleaned['AMT_ANNUITY'] > upper_bound) | (df_cleaned['AMT_ANNUITY'] < lower_bound) ].shape[0]

print(color.BOLD + color.BLUE + 'Lower bound for outliers : {0} & Upper bound : {1}'.format(lower_bound,upper_bound ) + color.END)

print(color.BOLD + color.BLUE + 'Total no of outliers for column {0} : {1}'.format(clmn,Total_no_of_outliers ) + color.END)

In [0]:
#calculate % of outliers in the data
100*Total_no_of_outliers/df_cleaned.shape[0]

In [0]:
#remove AMT_ANNUITY outliers from the dataset
df_cleaned = df_cleaned[(df_cleaned['AMT_ANNUITY'] <= upper_bound) & (df_cleaned['AMT_ANNUITY'] >= lower_bound )]

<a id = 'AMT_GOODS_PRICE'></a>

### Outlier analysis for AMT_GOODS_PRICE

In [0]:
enable_plotly_in_cell()
clmn = 'AMT_GOODS_PRICE'
df_cleaned['AMT_GOODS_PRICE'].iplot(kind="box", boxpoints = 'outliers', theme="white", title="AMT_GOODS_PRICE",
                                 yTitle='Price in $')





In [0]:
#Lets calculate the upper bound and lower bounds for outliers
q1, q3= np.percentile(df_cleaned['AMT_GOODS_PRICE'].dropna(),[25,75])
iqr = q3 - q1
lower_bound = q1 -(1.5 * iqr) 
upper_bound = q3 +(1.5 * iqr) 

In [0]:

Total_no_of_outliers = df_cleaned[(df_cleaned['AMT_GOODS_PRICE'] > upper_bound) | (df_cleaned['AMT_GOODS_PRICE'] < lower_bound) ].shape[0]

print(color.BOLD + color.BLUE + 'Lower bound for outliers : {0} & Upper bound : {1}'.format(lower_bound,upper_bound ) + color.END)

print(color.BOLD + color.BLUE + 'Total no of outliers for column {0} : {1}'.format(clmn,Total_no_of_outliers ) + color.END)

In [0]:
#calculate % of outliers in the data
100*Total_no_of_outliers/df_cleaned.shape[0]

In [0]:
#remove AMT_GOODS_PRICE outliers from the dataset
df_cleaned = df_cleaned[(df_cleaned['AMT_GOODS_PRICE'] <= upper_bound) & (df_cleaned['AMT_GOODS_PRICE'] >= lower_bound )]

<a id = 'CNT_FAM_MEMBERS'></a>

### Outlier analysis for CNT_FAM_MEMBERS

In [0]:
enable_plotly_in_cell()
clmn = 'CNT_FAM_MEMBERS'
df_cleaned['CNT_FAM_MEMBERS'].iplot(kind="box", boxpoints = 'outliers', theme="white", title="CNT_FAM_MEMBERS",
                                 yTitle='No of members')



In [0]:
#Lets calculate the upper bound and lower bounds for outliers
q1, q3= np.percentile(df_cleaned['CNT_FAM_MEMBERS'].dropna(),[25,75])
iqr = q3 - q1
lower_bound = round(q1 -(1.5 * iqr)) 
upper_bound = round(q3 +(1.5 * iqr)) 

In [0]:

Total_no_of_outliers = df_cleaned[(df_cleaned['CNT_FAM_MEMBERS'] > upper_bound) | (df_cleaned['CNT_FAM_MEMBERS'] < lower_bound) ].shape[0]

print(color.BOLD + color.BLUE + 'Lower bound for outliers : {0} & Upper bound : {1}'.format(lower_bound,upper_bound ) + color.END)

print(color.BOLD + color.BLUE + 'Total no of outliers for column {0} : {1}'.format(clmn,Total_no_of_outliers ) + color.END)


In [0]:
#calculate % of outliers in the data
100*Total_no_of_outliers/df_cleaned.shape[0]

In [0]:
#remove AMT_ANNUITY outliers from the dataset
df_cleaned = df_cleaned[(df_cleaned['CNT_FAM_MEMBERS'] <= upper_bound) & (df_cleaned['CNT_FAM_MEMBERS'] >= lower_bound )]

<a id = 'Binning'></a>

### Binning continous variables

<a id = 'DAYS_BIRTH'></a>

### DAYS_BIRTH

**We can create categories with age in years for DAYS_BIRTH variable. which can help us for futher analysis.**

In [0]:
def age(days):
    years = days/365
    
    if years <30:
        return '20+'
    elif (years >=30 and years < 40):
        return '30+'
    elif (years >=40 and years < 50):
        return '40+'
    elif (years >=50 and years < 60):
        return '50+'
    elif (years >=60 and years < 70):
        return '60+'
    else:
        return '70+'

In [0]:

df_cleaned['Age_Years'] =  df_cleaned['DAYS_BIRTH'].apply(lambda x:age(x))

In [0]:
sns.countplot(df_cleaned['Age_Years'])
plt.show()

<a id = 'DAYS_EMPLOYED'></a>

### DAYS_EMPLOYED

**We can create employement period in years from DAYS_EMPLOYED variable. which can help us for futher analysis.**

**we have a value 365243 which is a default value for retired applicants , this should be also taken care of during this binning process**

In [0]:
def age(days):
    years = days/365
    
    if years == 0:
        return '0'
    elif (years >0 and years <= 5):
        return '0-5'
    elif (years > 5 and years <=10 ):
        return '5-10'
    elif (years > 10 and years <=15 ):
        return '10-15'
    elif (years > 15 and years <=20 ):
        return '15-20'
    elif (years > 20 and years <=25 ):
        return '20-25'
    elif (years > 25 and years <=30 ):
        return '25-30'
    elif (years > 30 and years <=35 ):
        return '30-35'
    elif (years > 35 and years <=40 ):
        return '35-40'
    elif (years > 40 and years <=45 ):
        return '40-45'
    elif (years > 45 and years < 999 ):
        return '45+'
    else:
        return 'Retired'

In [0]:
df_cleaned['Employement_years'] =  df_cleaned['DAYS_EMPLOYED'].apply(lambda x:age(x))

In [0]:
sns.countplot(df_cleaned['Employement_years'])
plt.xticks(rotation=45)
plt.show()

In [0]:
##As the DAYS_EMPLOYED should be 365243 can affect future analysis, we can replace this value with null

df_cleaned.replace(365243,np.nan, inplace = True)

<a id = 'Export'></a>

## Export cleaned data to a csv file

In [0]:
df_cleaned.to_csv('Data_notebook1.csv')

## Continue further analysis on Notebook: 
Credit EDA Case Study-Part II.ipynb